In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [42]:
# URL of the validators page
url = "https://flaremetrics.io/validators"

# Send a GET request to the page
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the page content
    soup = BeautifulSoup(response.content, "html.parser")

    # Save soup as a text file
    with open('flare_metrics.txt', 'w', encoding='utf-8') as file:
        file.write(soup.prettify())
    
    # Find all validator entries
    validators = soup.find_all("tr", class_="h-16 border-y border-gray-100 dark:border-gray-600 text-sm font-medium md:hover:bg-gray-50 md:dark:hover:bg-gray-700")  # Adjust the class name if necessary
    
    # Initialize lists to store the data
    addresses = []
    uptimes = []
    fees = []
    delegators = []
    owner_stakes = []
    delegators_stakes = []
    free_spaces = []
    total_stakes = []
    
    for validator in validators:
        # Extract validator address
        address = validator.find("div", class_="text-ellipsis overflow-hidden text-xs opacity-70").get_text(strip=True)
        addresses.append(address)
        
        # Extract validator uptime
        uptime = validator.find("td", class_="text-left font-light pr-2").get_text(strip=True)
        uptimes.append(uptime)
        
        # Extract validator fee
        total_stake = validator.find("td", class_="text-left font-light").get_text(strip=True).split(":")[0][:-8]
        total_stake = total_stake.replace(',', '')
        total_stakes.append(total_stake)
        
        # Extract owner stake
        owner_stake = validator.find("div", string="Owner:").find_next("div", class_="text-right").get_text(strip=True).split()[0][:-3]
        owner_stake = owner_stake.replace(',', '')
        owner_stakes.append(owner_stake)
        
        # Extract delegators stake
        delegators_stake = validator.find("div", string="Delegators:").find_next("div", class_="text-right").get_text(strip=True).split()[0][:-3]
        delegators_stake = delegators_stake.replace(',', '')
        delegators_stakes.append(delegators_stake)
        
        # Extract free space
        free_space = validator.find("div", string="Free Space:").find_next("div", class_="text-right").get_text(strip=True).split()[0][:-3]
        free_space = free_space.replace(',', '')
        free_spaces.append(free_space)

        # Extract number of delegators
        delegators_count = validator.find("td", class_="relative text-left font-light").get_text(strip=True)[:-139]
        delegators_count = delegators_count.replace(',', '')
        delegators.append(delegators_count)

        # Extract fee
        fee = validator.find_all("td", class_="text-left font-light")[1].get_text(strip=True)[:-1]
        fees.append(fee)
    
    # Create a DataFrame
    data = {
        'Address': addresses,
        'Total Stake': total_stakes,
        'Owner Stake': owner_stakes,
        'Delegators Stake': delegators_stakes,
        'Free Space': free_spaces,
        'Delegators': delegators,
        'Uptime': uptimes,
        'Fee': fees
    }
    
    df = pd.DataFrame(data)
    
    # Save DataFrame to a CSV file
    df.to_csv('validators_data.csv', index=False)
    
    #print(df)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [43]:
df

,Address,Total Stake,Owner Stake,Delegators Stake,Free Space,Delegators,Uptime,Fee
0,8qSyua2WVQ4uNRc1wwPKmSUWY421bifiq,199949972,30000000,169949972,50028,375,100.00%,10.00
1,MfzBQ6MbriE31UWm9NjSnEgLiR1cAkEnN,199952410,29990100,169962310,47590,320,99.99%,10.00
2,A7jk7Z5GA2aVe5ecuupNXWtSLfVjLDWFC,199879725,30000000,169879725,120275,246,99.98%,7.50
3,EgJba99Q5iDcL81A27E111WANn9eZNj4v,49264614,3500000,45764614,3235386,210,100.00%,20.00
4,2hCHs6H4Vde8mdGGKidMvT4Fjit7goCBT,44999895,3000000,41999895,105,198,98.15%,2.00
...,...,...,...,...,...,...,...,...
89,JHnfNSNgSThSJEFG8aq99FRAE4a7Ko7UP,4000000,4000000,0,56000000,0,100.00%,20.00
90,CeX1NfNPHCdaJXUVtkLysShaZCtsHRK3D,2172490,2172490,0,30414860,0NO REWARDS,100.00%,20.00
91,KppZbRMtfNTQZGdmHdkbHTJHgNPBCNxJj,1300000,1300000,0,18200000,0,100.00%,22.00
92,J8Zbi3b3AjTEUmHdE4KSApEU1U36iaeDS,8000000,8000000,0,112000000,0,100.00%,18.00


In [45]:
df['Delegators'].to_numpy()

array(['375', '320', '246', '210', '198', '186', '170', '134', '122',
       '122', '105', '96', '91', '88', '86', '78', '78', '77', '77', '69',
       '65', '63', '62', '62', '60', '59', '56', '56', '55', '54', '51',
       '48', '45', '38', '38', '37', '37NO REWARDS', '37', '34', '33',
       '32', '30', '29', '28', '27', '26', '26NO REWARDS', '26', '24',
       '24', '24', '23', '23', '23', '21', '20', '19', '18', '17', '16',
       '15', '15', '15', '15', '15', '14NO REWARDS', '14', '14', '13',
       '13', '13', '13', '11NO REWARDS', '11NO REWARDS', '11', '10', '10',
       '8NO REWARDS', '8', '7', '7', '6', '3', '3', '2', '2', '2', '1',
       '1', '0', '0NO REWARDS', '0', '0', '0'], dtype=object)